In [ ]:
import os
import sys
import re
import importlib.util

In [ ]:
# variables
txts_path = "/datasets/txts" 
txts_cleaned_path = "outputs/txts_cleaned"

In [ ]:
!mkdir -p $txts_cleaned_path

In [ ]:
for txt_file in os.listdir(txts_path):
    txt_path = f"{txts_path}/{txt_file}"
    cleaned_file_path = f"{txts_cleaned_path}/{txt_file}"
    inputs = ""
    with open(txt_path) as infile:
        inputs = infile.read()
    inputs = inputs.split("\n\n")
    outputs = []
    for input in inputs:
        if not input or input.isspace():
            continue
        if re.match("^(BEGIN TRANSCRIPT:|END TRANSCRIPT)", input):
            outputs.append(input)
            continue
        if re.match("^https:", input):
            continue
        if re.search("\d{1,2}/\d{1,2}/\d{4}, \d{1,2}:\d{1,2}", input):
            continue
        if re.match("^\d+/\d+", input):
            continue
        input = input.replace("\n", " ")
        input = input.replace("(Pause):", " ")
        input = re.sub("\([^\)]*\)", " ", input)
        input = re.sub("\[[^\]]*\]", " ", input)
        if re.match("^(CLIENT:|PATIENT:|CL:)", input):
            input = re.sub("^(CLIENT:|PATIENT:|CL:)", "CLIENT:", input)
            outputs.append(input)
        elif re.match("^(THERAPIST:|COUNSELOR:|TL:)", input):
            input = re.sub("^(THERAPIST:|COUNSELOR:|TL:)", "THERAPIST:", input)
            outputs.append(input)
        elif len(outputs) > 0:
            last = outputs.pop()
            last = last + " " + input
            outputs.append(last)

    output = "\n".join(outputs)
    with open(cleaned_file_path, "w") as outfile:
        outfile.write(output)
    print(f"Cleaned up file {txt_file} and written")

In [ ]:
!ls $txts_cleaned_path